In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('ship_crew_model').getOrCreate()

In [3]:
df=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True) 

In [4]:
df.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [7]:
from pyspark.ml.feature import StringIndexer 
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat') 
indexed=indexer.fit(df).transform(df)

In [8]:
for item in indexed.head(5): 
    print(item) 
    print('\n') 

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [9]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler=VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'cruise_cat'],
                          outputCol='features') 
output=assembler.transform(indexed) 
output.select('features','crew').show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [11]:
final_data=output.select('features','crew') 
train_data,test_data=final_data.randomSplit([0.7,0.3]) 
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               113|
|   mean| 7.782300884955763|
| stddev|3.3701817481855167|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [12]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                45|
|   mean| 7.824000000000001|
| stddev|3.8580426030919774|
|    min|               0.6|
|    max|              21.0|
+-------+------------------+



In [13]:
from pyspark.ml.regression import LinearRegression
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')
trained_ship_model=ship_lr.fit(train_data)
ship_results=trained_ship_model.evaluate(train_data)
print('Rsquared Error :',ship_results.r2)

Rsquared Error : 0.9187225879512978


In [14]:
unlabeled_data=test_data.select('features') 
unlabeled_data.show(5) 

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,122.0,28.5,1...|
|[5.0,133.5,39.59,...|
|[6.0,30.276999999...|
|[6.0,93.0,23.94,9...|
+--------------------+
only showing top 5 rows



In [15]:
predictions=trained_ship_model.transform(unlabeled_data) 
predictions.show() 

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.112314511044737|
|[5.0,122.0,28.5,1...| 6.421119591770415|
|[5.0,133.5,39.59,...|13.116103593759918|
|[6.0,30.276999999...| 4.268881882991163|
|[6.0,93.0,23.94,9...|10.469966187458274|
|[6.0,113.0,37.82,...|11.617088638282443|
|[9.0,85.0,19.68,9...| 9.277092336349135|
|[9.0,88.5,21.24,9...| 9.483491495888249|
|[10.0,86.0,21.14,...| 9.613321879199239|
|[10.0,105.0,27.2,...|11.270394949535818|
|[10.0,110.0,29.74...|   12.076214674469|
|[11.0,85.0,18.48,...| 8.724949054216438|
|[11.0,91.0,20.32,...| 9.191117950041447|
|[12.0,2.329,0.94,...| 0.591638734642524|
|[12.0,25.0,3.88,5...| 2.908254152732823|
|[12.0,50.0,7.0,7....| 4.493409032370467|
|[12.0,77.104,20.0...| 8.742535446586952|
|[12.0,91.0,20.32,...| 9.187562311475352|
|[12.0,91.0,20.32,...| 9.187562311475352|
|[12.0,108.865,27....| 10.86093770853874|
+--------------------+------------